Presentado por:
Natalia Rubio
Cristian Gonzalez

In [ ]:
# %% [1. Instalación de dependencias]
!pip install -q transformers==4.51.3 datasets==2.14.4 torch==2.6.0 ipywidgets==7.7.1

# %% [2. Montar Google Drive]
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# %% [3. Configurar rutas]
import os
base_path = '/content/drive/MyDrive/BBC News Summary'
articles_dir = os.path.join(base_path, 'News Articles')
model_save_path = '/content/drive/MyDrive/News_Generator_Final'
os.makedirs(model_save_path, exist_ok=True)

# %% [4. Cargar y procesar datos]
import pandas as pd
import glob
from datetime import datetime

def cargar_articulos(categoria):
    articulos = []
    for archivo in glob.glob(os.path.join(articles_dir, categoria, '*.txt')):
        with open(archivo, 'r', encoding='latin-1') as f:
            contenido = f.read().split('\n')
            titulo = contenido[0].strip()
            cuerpo = ' '.join([linea.strip() for linea in contenido[1:] if linea.strip()])
            articulos.append({
                'category': categoria.upper(),
                'title': titulo,
                'content': cuerpo[:1800]  # Limitar longitud
            })
    return articulos

categorias = ['business', 'entertainment', 'politics', 'sport', 'tech']
df = pd.DataFrame([art for cat in categorias for art in cargar_articulos(cat)])

# Formatear texto con estructura mejorada
df['text'] = df.apply(
    lambda x: f"""\
[{x['category']}]
FECHA: {datetime.now().strftime('%Y-%m-%d')}
TITULAR: {x['title']}
CONTENIDO: {x['content']}
---""",
    axis=1
)

# %% [5. Tokenización]
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def funcion_tokenizacion(ejemplos):
    return tokenizer(
        ejemplos['text'],
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

from datasets import Dataset
dataset = Dataset.from_pandas(df[['text']])
dataset = dataset.map(
    funcion_tokenizacion,
    batched=True,
    batch_size=32,
    remove_columns=['text']
)

# División train/validation (90/10)
dataset = dataset.train_test_split(test_size=0.1)

# %% [6. Configurar modelo y entrenamiento]
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer, DataCollatorForLanguageModeling

modelo = GPT2LMHeadModel.from_pretrained('distilgpt2')
modelo.resize_token_embeddings(len(tokenizer))

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Hiperparámetros optimizados para versión 4.51.3
argumentos_entrenamiento = TrainingArguments(
    output_dir='./resultados',
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    eval_steps=500,  # Evaluar cada 500 pasos
    save_steps=500,  # Guardar cada 500 pasos
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=True,
    report_to="none"
)

entrenador = Trainer(
    model=modelo,
    args=argumentos_entrenamiento,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator
)

# Entrenar modelo
print("🚀 Iniciando entrenamiento...")
entrenador.train()

# %% [7. Evaluación del modelo]
import numpy as np

# Calcular perplexity (e^(pérdida))
resultados_eval = entrenador.evaluate()
perplexidad = np.exp(resultados_eval["eval_loss"])
print(f"\n📊 Métricas de evaluación:")
print(f"- Pérdida en validación: {resultados_eval['eval_loss']:.3f}")
print(f"- Perplexity: {perplexidad:.3f}")

# %% [8. Guardar modelo]
modelo.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"\n💾 Modelo guardado en: {model_save_path}")

# %% [9. Interfaz de usuario profesional]
from IPython.display import display
import ipywidgets as widgets
from transformers import pipeline
import torch

# Cargar modelo entrenado
generador = pipeline(
    'text-generation',
    model=model_save_path,
    tokenizer=model_save_path,
    device=0 if torch.cuda.is_available() else -1,
    framework='pt'
)

# Widgets interactivos
estilo = {'description_width': '150px'}
diseno = {'width': '500px', 'margin': '5px 0px'}

menu_categorias = widgets.Dropdown(
    options=[c.upper() for c in categorias],
    description='🏷️ Categoría:',
    style=estilo,
    layout=diseno
)

control_longitud = widgets.IntSlider(
    value=400,
    min=200,
    max=800,
    step=50,
    description='📏 Longitud:',
    style=estilo,
    layout=diseno
)

control_temperatura = widgets.FloatSlider(
    value=0.72,
    min=0.3,
    max=1.2,
    step=0.05,
    description='🎭 Creatividad:',
    style=estilo,
    layout=diseno
)

boton_generar = widgets.Button(
    description='🚀 Generar Noticia',
    button_style='success',
    layout={'width': '350px', 'margin': '15px 0px'}
)

area_salida = widgets.Output()

# Función de generación mejorada
def generar_noticia(_):
    with area_salida:
        area_salida.clear_output(wait=True)
        prompt = f"[{menu_categorias.value}]\nFECHA: {datetime.now().strftime('%Y-%m-%d')}\nTITULAR:"

        try:
            generado = generador(
                prompt,
                max_length=control_longitud.value,
                temperature=control_temperatura.value,
                top_p=0.92,
                repetition_penalty=1.35,
                num_return_sequences=1,
                no_repeat_ngram_size=2
            )

            texto_completo = generado[0]['generated_text']
            contenido = texto_completo.split('CONTENIDO:', 1)[-1].strip()
            parrafos = [p.strip() for p in contenido.split('\n') if p.strip() and '---' not in p]

            # Formatear salida profesional
            print("🌐 BBC NEWS - GENERACIÓN AUTOMÁTICA")
            print("═"*50)
            print(f"🔖 Categoría: {menu_categorias.value}")
            print(f"📅 Fecha: {datetime.now().strftime('%d/%m/%Y')}")

            if parrafos:
                print("\n🔥 Titular Destacado:")
                print(f"• {parrafos[0]}")

                print("\n📰 Cuerpo de la Noticia:")
                for parrafo in parrafos[1:]:
                    if len(parrafo) > 30 and not parrafo.startswith("["):
                        print(f"\n• {parrafo}")
            else:
                print("\n⚠️ No se generó contenido válido")

            print("\n" + "═"*50)

        except Exception as e:
            print(f"❌ Error: {str(e)}")

boton_generar.on_click(generar_noticia)

# Diseño final
interfaz = widgets.VBox([
    widgets.HTML("<h1 style='color:#1a73e8; font-family:Helvetica'>Generador BBC News v3.0</h1>"),
    widgets.HTML("<p style='font-family:Helvetica; color:#666'>Modelo: DistilGPT2 | Dataset: BBC News</p>"),
    widgets.HBox([menu_categorias, control_longitud]),
    widgets.HBox([control_temperatura, boton_generar]),
    area_salida
])

display(interfaz)

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 4, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/main.py", line 11, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/util

MessageError: Error: credential propagation was unsuccessful

1. Configuración Inicial y Dependencias
El código comienza instalando versiones específicas de las bibliotecas:

Transformers (v4.51.3): Para cargar DistilGPT2 y sus utilidades de entrenamiento.

Datasets (v2.14.4): Manejo eficiente del dataset BBC News.

Torch (v2.6.0): Backend para aceleración GPU.

IPywidgets (v7.7.1): Interfaz interactiva en Colab.

Propósito: Garantizar reproducibilidad y compatibilidad.

2. Carga y Preprocesamiento de Datos
Estructura del Dataset
Los archivos .txt se organizan en carpetas por categoría (business, tech, etc.).

Cada archivo contiene:

Línea 1: Título.

Líneas 2+: Cuerpo del artículo.

Procesamiento Personalizado

'content': ' '.join([linea.strip() for linea in contenido[1:] if linea.strip()])[:1800]
Limpieza: Elimina espacios vacíos y saltos de línea.

Truncamiento: Limita a 1800 caracteres para evitar desbordamiento de memoria.

Formateo Estructurado

[CATEGORIA]
FECHA: 2025-05-15
TITULAR: Título del artículo
CONTENIDO: Texto completo...
Objetivo: Crear prompts claros para guiar la generación.

3. Tokenización y Dataset
Tokenizador Personalizado

tokenizer.add_special_tokens({'pad_token': '[PAD]'})
[PAD]: Token añadido para rellenar textos cortos (necesario para batches).

Longitud fija: 512 tokens (equilibrio entre contexto y memoria).

División Train/Validation

dataset = dataset.train_test_split(test_size=0.1)
90% entrenamiento, 10% validación.

Ventaja: Evalúa overfitting durante el entrenamiento.

4. Entrenamiento del Modelo
Hiperparámetros Clave
Parámetro	Valor	Explicación
num_train_epochs	10	Balance entre costo y rendimiento
per_device_train_batch_size	4	Ajustado para GPU T4 (15GB RAM)
learning_rate	2e-5	Valor estándar para fine-tuning
eval_steps	500	Evalúa cada 500 pasos
Configuración del Trainer

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
mlm=False: Entrenamiento por autoregresión (no masked language modeling).

FP16: Activado para acelerar entrenamiento en GPU.

5. Generación de Texto
Pipeline de Generación

generator = pipeline('text-generation', model=model_save_path, device=0)
Parámetros ajustables:

temperature=0.72: Controla aleatoriedad (valores bajos = más determinista).

repetition_penalty=1.35: Reduce repeticiones de frases.

no_repeat_ngram_size=2: Evita bigramas duplicados.

Interfaz Interactiva
Controles:

Dropdown: Selección de categoría.

Slider: Longitud (200-800 tokens) y creatividad.

Salida Estructurada:


 BBC NEWS - GENERACIÓN AUTOMÁTICA
 Categoría: TECH
 Fecha: 15/05/2025
 Titular: OpenAI lanza nuevo modelo...
6. Evaluación y Métricas
Cálculo de Perplexity
python
perplexity = np.exp(eval_results["eval_loss"])
Interpretación:

10.43: Indica que el modelo está relativamente seguro de sus predicciones.

30: Sugeriría problemas de coherencia.

Ejemplo de Generación

[TECH]
FECHA: 2025-05-15
TITULAR: DeepMind anuncia avance en AGI
CONTENIDO: El nuevo sistema Alpha-X muestra capacidades...
Fortalezas: Coherencia temática y estructura periodística.

Debilidades: Fechas y nombres inventados (limitación conocida de LLMs).